In [1]:
# Load the required packages
import time
import psutil
import numpy as np
import pandas as pd
import glob
import multiprocessing as mp

# For data visualizatoin
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot
from bokeh.io import output_notebook, show
from bokeh.models import Title
from bokeh.plotting import figure, output_file, show

from datetime import datetime, timedelta

import dask.dataframe as dd


## Removing unwanted data from RAW weather dataset 
- Read all the MIDAS weather observation files from 2011-2014 
- Add column headers from separate txt file
- Remove unwanted columns (listed in other txt file)
- Rename the columns
- Save the dataframe to multiple(Dask thing, can't help!) CSV files 

For more information about dataset parameters, visit:

`https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/WH_Table.html`

In [11]:
path=r'./midas_wxhrly' # relative path to weather data files
# files= glob.glob(path + "/*.txt")
# print(files)

df_ = dd.read_csv(path+'/*.txt', dtype='str') #read all the midas weather files 
df_header = dd.read_csv(path+'/columns/columns.txt', header=0) # List of all columns -> 104 parameters
df_col_del = pd.read_csv(path+'/columns/columns_delete.txt') # Columns to be deleted are supplied from this txt file
df_.columns = df_header.columns

df = df_.drop(df_col_del.columns, axis=1)
df.columns = ['GMT', 'VERSION_NUM', 'SRC_ID' ,'WIND_DIRECTION' ,'WIND_SPEED', 'VISIBILITY', 'MSL_PRESSURE', 
              'AIR_TEMPERATURE', 'DEWPOINT', 'WETB_TEMP', 'STN_PRES', 'WMO_HR_SUN_DUR']
df_GMT = df['GMT']
df = df.applymap(lambda x: x.replace(' ', '')) # For smooth str-to-float conversion in next step
df = df.drop('GMT', axis =1)
df['GMT'] = df_GMT
df.to_csv('./mod_datasets/weather_all_stations/weather-*.csv', sep=',')

['./mod_datasets/weather_all_stations/weather-00.csv',
 './mod_datasets/weather_all_stations/weather-01.csv',
 './mod_datasets/weather_all_stations/weather-02.csv',
 './mod_datasets/weather_all_stations/weather-03.csv',
 './mod_datasets/weather_all_stations/weather-04.csv',
 './mod_datasets/weather_all_stations/weather-05.csv',
 './mod_datasets/weather_all_stations/weather-06.csv',
 './mod_datasets/weather_all_stations/weather-07.csv',
 './mod_datasets/weather_all_stations/weather-08.csv',
 './mod_datasets/weather_all_stations/weather-09.csv',
 './mod_datasets/weather_all_stations/weather-10.csv',
 './mod_datasets/weather_all_stations/weather-11.csv',
 './mod_datasets/weather_all_stations/weather-12.csv',
 './mod_datasets/weather_all_stations/weather-13.csv',
 './mod_datasets/weather_all_stations/weather-14.csv',
 './mod_datasets/weather_all_stations/weather-15.csv',
 './mod_datasets/weather_all_stations/weather-16.csv',
 './mod_datasets/weather_all_stations/weather-17.csv',
 './mod_da

In [12]:
df['GMT']

Dask Series Structure:
npartitions=94
    object
       ...
     ...  
       ...
       ...
Name: GMT, dtype: object
Dask Name: getitem, 1034 tasks

## Make final weather dataset
- Load all the weather files stored from previous cell
- Select `VERSION_NUM=1` and best possible weather station(s)
- Remove columns`('VERSION_NUM', 'SRC_ID', 'Unnamed: 0')`........ `'Unnamed: 0'`- dataframe index before making timestamp an index
- Save the dataframe in CSV format

For information on weather stations:

`http://archive.ceda.ac.uk/midas_stations/`

In [13]:
path = r'./mod_datasets/weather_all_stations'

parse_dates = ['GMT']
dtypes={ 'VERSION_NUM':'float', 'SRC_ID':'float', 'WIND_DIRECTION':'float', 'WIND_SPEED':'float', 'VISIBILITY':'float', 
        'MSL_PRESSURE':'float', 'AIR_TEMPERATURE':'float', 'DEWPOINT':'float', 'WETB_TEMP':'float', 'STN_PRES':'float', 'WMO_HR_SUN_DUR':'float'}

ddf = dd.read_csv(path+'/*.csv', sep=',', dtype=dtypes, parse_dates=parse_dates)

ddf = ddf[ddf.VERSION_NUM==1] # Verified data entries

ddf = ddf[ddf.SRC_ID==708] # Heathrow observatory ID.... more info: http://archive.ceda.ac.uk/midas_stations/

ddf = ddf.set_index('GMT')

# Remove the unwanted columns
col_list=['VERSION_NUM', 'SRC_ID', 'Unnamed: 0']
ddf = ddf.drop(col_list, axis=1)

# Save the dataframe in CSV format
ddf.to_csv('./mod_datasets/weather_final_2011_2014/midas_weather-*.csv', sep=',')
ddf.tail(4)

,WIND_DIRECTION,WIND_SPEED,VISIBILITY,MSL_PRESSURE,AIR_TEMPERATURE,DEWPOINT,WETB_TEMP,STN_PRES,WMO_HR_SUN_DUR
GMT,,,,,,,,,
2014-12-31 20:00:00,190.0,8.0,1000.0,1033.2,6.1,4.4,5.4,1030.1,0.0
2014-12-31 21:00:00,190.0,9.0,1400.0,1033.1,6.5,4.2,5.5,1029.9,0.0
2014-12-31 22:00:00,190.0,10.0,1400.0,1032.5,5.7,3.0,4.6,1029.3,0.0
2014-12-31 23:00:00,190.0,8.0,1300.0,1032.7,5.0,2.4,3.9,1029.5,0.0


## Graphs Time!!!

In [24]:
path = r'./mod_datasets/weather_final_2011_2014'
allFiles = glob.glob(path + "/*.csv")
allFiles = sorted(allFiles)

parse_dates = ['GMT']
dtypes={'GMT':'str', 'WIND_DIRECTION':'float', 'WIND_SPEED':'float', 'VISIBILITY':'float', 
        'MSL_PRESSURE':'float', 'AIR_TEMPERATURE':'float', 'DEWPOINT':'float', 'WETB_TEMP':'float', 'STN_PRES':'float', 'WMO_HR_SUN_DUR':'float'}

# ddf = dd.read_csv(path+'/*.csv', sep=',', dtype=dtypes, parse_dates=parse_dates)
# ddf.tail(4)

list_ = []
for file_ in allFiles:
    df_1 = pd.read_csv(file_,index_col=None, header=0, dtype=dtypes, parse_dates = parse_dates)
    list_.append(df_1)
    
    
df = pd.concat(list_, axis = 0, ignore_index=True)
df = df.set_index('GMT')
print(sum(df.duplicated()))
df = df.drop_duplicates()
df.to_csv('./mod_datasets/midas_weather.csv', sep=',')


26303


In [23]:
df.head(4)

,WIND_DIRECTION,WIND_SPEED,VISIBILITY,MSL_PRESSURE,AIR_TEMPERATURE,DEWPOINT,WETB_TEMP,STN_PRES,WMO_HR_SUN_DUR
GMT,,,,,,,,,
2011-01-01 00:00:00,320.0,3.0,900.0,1025.8,5.9,2.6,4.5,1022.7,NaN
2011-01-01 01:00:00,310.0,5.0,1000.0,1025.4,5.8,2.5,4.4,1022.3,0.0
2011-01-01 02:00:00,300.0,4.0,800.0,1025.4,5.5,2.7,4.4,1022.2,0.0
2011-01-01 03:00:00,300.0,5.0,800.0,1025.2,5.4,2.7,4.3,1022.0,0.0


In [ ]:
# ONLY for dask dataframe

ym_group = ddf.groupby(df.GMT.dt.hour).value.mean().compute()

ym_df=ym_group.compute()

plt.figure(figsize=(12,8))
ym_df.unstack().T.sum().plot()  # Plot publications by year
plt.title('Articles published per year')

In [19]:
# Plot the usage pattern during the trial



output_notebook()
# output_file("./temp/line.html") #Uncomment it to save the plot in html file
p=figure(x_axis_type = 'datetime', plot_width=800, plot_height=400)
p.line(df.index, df.loc[:,'AIR_TEMPERATURE'].values, line_width=1)
# p.line(df_aggregate.index, df_aggregate.loc[:,"trial_n"], line_width=1, color='red')

show(p)

Loading BokehJS ...